In [8]:
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

consumer_key = 'Umjx7sONzSvEBOSjKKvuyDMy1'
consumer_secret = '9E5wiXG9Uj2x9e4wOnKX4j2HXFIAFoNi5pmnNl4IfJRpNHcIpt'
access_token = '122583337-xrUwHzBEHYlRidzit1Pmjt3so3ucEnYw9JnAqOjB'
access_token_secret = 'yLmlazDOrxwZdu6eDHZAjSewivdkOdZaqr24AAjV04jCo'


def get_twitter():
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.
    """
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)

def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            sys.stderr.flush()
            time.sleep(61 * 15)
            
def get_friends(twitter, screen_name):
    """
    This function will return the friends on the twitter, w.r.t. the screen_name.
    Input: twitterApi instance, screen_name 
    Output: Returns the friends w.r.t the screen name.
    """
    send=list()
    string= screen_name+'&count=5000'
    response=robust_request(twitter,"friends/list",{'screen_name':screen_name,'count':'200'},5)
    for r in response:
        send.append(r['screen_name'])
    return send

def dataCollection():
    """
    It collects the data to be used for the other methods like classify, and cluster.
    Here we are taking four locations and the latest tweets w.r.t the hashtag is stored in a csv file for sentiment analysis.
    Also, a csv file for clustering is also generated.
    """
    twitter = get_twitter()
    #coords = dict()
    users = list()
    data = {}
    #dictionary with the coordinates of the places.
    coords = {"Chicago":('41.8781,-87.6298,50mi'),"London":('51.5074,-0.1278,50mi'),"Sydney":('36.778261,-119.41793239999998,50mi')}
    
    
    for key, value in coords.items():
        ulist = list()
        i = 0
        res = list()
        response=robust_request(twitter,"search/tweets",{'q':'#JusticeLeague','count':'200','geocode':value},5)
        for r in response:
            res.append(r)
            if not r['user']['screen_name'] in ulist:
                if i < 5:
                    ulist.append(r['user']['screen_name'])
                users.append(r['user']['screen_name'])
            else:
                continue
            i = i +1
                
        #dictionary with key as the places and value as the screenname of the person who tweeted with the hashtag(first five).
        data[key]=ulist
        #To create a csv file w.r.t the location, we will use pandas dataframe.
        frame = pd.DataFrame(res)
        #fname=key+"_classify.csv"
        frame.to_csv(key+"-classify.csv",sep='\t')
    print(data)
       
    u=np.unique(np.array(users))
    f = open("collect.txt", 'w+')
    if f.mode=='w+':
        f.write("Number of users Collected:"+str(len(u)))
        f.write("\nNumber of messages Collected:"+str(len(users)))
        f.close()
    
    
    #csv file for later stage like clustering is created w.r.t the place. 
    for city,user in data.items():
        cols = ['Id', 'friends']
        newDataFrame = pd.DataFrame(columns=cols)
        for eachuser in user:
            len_of_nd = len(newDataFrame)
            newDataFrame.loc[len_of_nd]=[eachuser,get_friends(twitter,eachuser)]
        newDataFrame.to_csv(city+"-cluster.csv",sep='\t')
    
    
    
    
def main():
    """ Main method. You should not modify this. """
    dataCollection()

if __name__ == '__main__':
    main()

{'Chicago': ['Vermouth1991', 'caleb_bandy', 'HollarDollar', 'austinelee_', 'OsvaldoMMFilho'], 'London': ['xChristinaPath', 'marijacobson13', 'parkperrie', 'TaijaTalyssa92', 'salutebellarke'], 'Sydney': ['HankRea', 'TanmayDC777', 'BrianIpuche28', 'JeremyGuevara17', 'ClaudeRescu']}
